# **1767. Find the Subtasks That Did Not Execute**
``` console
Table: Tasks

+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| task_id        | int     |
| subtasks_count | int     |
+----------------+---------+
task_id is the column with unique values for this table.
Each row in this table indicates that task_id was divided into subtasks_count subtasks labeled from 1 to subtasks_count.
It is guaranteed that 2 <= subtasks_count <= 20.
 

Table: Executed

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| task_id       | int     |
| subtask_id    | int     |
+---------------+---------+
(task_id, subtask_id) is the combination of columns with unique values for this table.
Each row in this table indicates that for the task task_id, the subtask with ID subtask_id was executed successfully.
It is guaranteed that subtask_id <= subtasks_count for each task_id.
 

Write a solution to report the IDs of the missing subtasks for each task_id.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Tasks table:
+---------+----------------+
| task_id | subtasks_count |
+---------+----------------+
| 1       | 3              |
| 2       | 2              |
| 3       | 4              |
+---------+----------------+

Executed table:
+---------+------------+
| task_id | subtask_id |
+---------+------------+
| 1       | 2          |
| 3       | 1          |
| 3       | 2          |
| 3       | 3          |
| 3       | 4          |
+---------+------------+

Output: 
+---------+------------+
| task_id | subtask_id |
+---------+------------+
| 1       | 1          |
| 1       | 3          |
| 2       | 1          |
| 2       | 2          |
+---------+------------+

Explanation: 
Task 1 was divided into 3 subtasks (1, 2, 3). Only subtask 2 was executed successfully, so we include (1, 1) and (1, 3) in the answer.
Task 2 was divided into 2 subtasks (1, 2). No subtask was executed successfully, so we include (2, 1) and (2, 2) in the answer.
Task 3 was divided into 4 subtasks (1, 2, 3, 4). All of the subtasks were executed successfully.

```

**Solution-1:**

``` sql
# Write your MySQL query statement below
WITH RECURSIVE cte as(
    SELECT 
    1 AS task_id, 1 AS subtasks_count
    UNION 
    SELECT
    task_id + 1, subtasks_count
    FROM CTE
    WHERE task_id <(SELECT max(task_id) FROM TASKS)
    UNION 
    SELECT 
    task_id, 1 + subtasks_count
    FROM CTE
    where subtasks_count < 20
)
SELECT a.task_id, a.subtask_id
FROM
(SELECT a.task_id as task_id, a.subtasks_count AS subtask_id
FROM cte a, tasks b
WHERE a.task_id = b.task_id and a.subtasks_count <= b.subtasks_count) a
WHERE (a.task_id,a.subtask_id) NOT IN (SELECT task_id,subtask_id FROM executed)

**Solution-2:**

``` sql
with recursive number as(
    select * from Tasks
    union
    select task_id, subtasks_count  - 1
    from number 
    where subtasks_count  > 1
)
select a.task_id, a.subtasks_count as subtask_id
from number a
WHERE (a.task_id,a.subtasks_count) NOT IN (SELECT task_id,subtask_id FROM executed)

**Solution-3:**

``` sql
with recursive number as(
    select *
        from Tasks
    union
    select task_id, subtasks_count  - 1
        from number where subtasks_count  > 1
)
select a.task_id,a.subtasks_count as subtask_id
from number a
LEFT JOIN executed b
ON a.task_id = b.task_id AND a.subtasks_count = b.subtask_id 
WHERE b.task_id is null